In [1]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
from motion_generation.MoE_Z import MoE as MoE_Z
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.LSTM import LSTM
from motion_generation.MotionGeneration import MotionGenerationModel
from motion_generation.MotionGeneration_v2 import MotionGenerationModel as MotionGenerationModel_v2
from motion_generation.MotionGeneration_v3 import MotionGenerationModel as MotionGenerationModel_v3
from motion_generation.MotionGeneration_v4 import MotionGenerationModel as MotionGenerationModel_v4
from motion_generation.MotionGeneration_v4_Concat import MotionGenerationModel as MotionGenerationModel_Concat
from motion_generation.MotionGenerationVAE_v4 import MotionGenerationModel as MotionGenerationModel_VAE
from motion_generation.MotionGenerationVAE_Z import MotionGenerationModel as MotionGenerationModel_VAE_ZZ
from motion_generation.MotionGenerationVAE_Z_everywhere import MotionGenerationModel as MotionGenerationModel_VAE_Z
from motion_generation.MotionGenerationRNN import MotionGenerationModelRNN
from motion_generation.MotionGenerationBatch import MotionGenerationModelBatch
from rig_agnostic_encoding.models.MLP import MLP
from rig_agnostic_encoding.models.MLP_v2 import MLP as MLP_v2

from rig_agnostic_encoding.models.DEC import DEC
from rig_agnostic_encoding.models.MLP_MIX import MLP_MIX
from rig_agnostic_encoding.models.MLP_MIX_cosine import MLP_MIX as MLP_MIX_COSINE

from rig_agnostic_encoding.models.RBF import RBF
import rig_agnostic_encoding.models.RBF as RB
from rig_agnostic_encoding.models.RBF_Ortho import RBF as RBF_ORTHO
from rig_agnostic_encoding.models.RBF_Ortho_v2 import RBF as RBF_ORTHO_v2


from rig_agnostic_encoding.models.VAE import VAE
from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
from GlobalSettings import MODEL_PATH
import bz2
from cytoolz import concat, sliding_window, accumulate
from operator import add
from collections import OrderedDict
import torch
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib
import random

In [29]:
config = {
    "hidden_dim": 256,
    "k": 256,
    "z_dim": 256,
    "lr": 1e-4,
    "batch_size": 16,
    "keep_prob": .2,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.AdamW,
    "scheduler":torch.optim.lr_scheduler.StepLR,
    "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":64,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0,
    "autoregress_inc":0.3,
    "autoregress_ep":20,
    "autoregress_max_prob":1,
    "cost_hidden_dim":128,
    "seq_len":13,
    "device":"cuda"
    }

In [7]:
def getFilesNames(file_paths, data_path, MAX_FILES=-1):
    for dname, dirs, files in os.walk(data_path):
        for i, file in enumerate(files):
            file_paths.append(os.path.join(dname, file))
            if MAX_FILES > 0 and i >= MAX_FILES:
                break
    return file_paths

data_path = "/home/nuoc/Documents/MEX/data/data/Dataset_R1_Two_1"
data_path2 = "/home/nuoc/Documents/MEX/data/data/Dataset_R2_Two_1"
# data_path3 = "/home/nuoc/Documents/MEX/data/data/Dataset_R3_Two_1"
# data_path4 = "/home/nuoc/Documents/MEX/data/data/Dataset_R4_Two_1"
file_paths = getFilesNames([],data_path)
file_paths2 = getFilesNames([],data_path2)
# file_paths3 = getFilesNames([],data_path3)
# file_paths4 = getFilesNames([],data_path4)

print(len(file_paths))
print(len(file_paths2))





240
240


In [8]:
phase_features = ["phase_vec_l2"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost"]
pose_label_feature = ["chainPos", "isLeft", "geoDistanceNormalised"]
target_features = ["targetPosition", "targetRotation"]
features = phase_features + pose_features + cost_features + target_features
clips = []
feature_dims = {}


In [9]:
data = F.process_data_multithread(file_paths, features)
data_10 = F.process_data_multithread(file_paths, features, level=1)
data2 = F.process_data_multithread(file_paths2, features)
data2_10 = F.process_data_multithread(file_paths2, features, level=1)

2021-05-10 17:37:20,737	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-10 17:37:41,724	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-10 17:38:01,229	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-10 17:38:21,635	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [10]:
pose_labels1 = F.process_data_multithread([file_paths[0]], pose_label_feature)
pose_labels2 = F.process_data_multithread([file_paths2[0]], pose_label_feature)

2021-05-10 17:38:41,447	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-10 17:38:46,398	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [11]:
obj = {"data":data, "data2":data2, "pose_label1":pose_labels1, "pose_label2":pose_labels2}
F.save(obj, filename="transfer_learning_set_R1-R2_Two_upscaling", path="/home/nuoc/Documents/MEX/data/")

In [3]:
obj = F.load("/home/nuoc/Documents/MEX/data/transfer_learning_set_R1-R5_Two.pbz2")
data = obj["data"]
data2 = obj["data2"]

In [4]:
pose_labels1 = obj["pose_label1"]
pose_labels2 = obj["pose_label2"]

In [17]:
feature_dims = data[0][1]
feature_dims2 = data2[0][1]
feature_dims_10 = data_10[0][1]
feature_dims2_10 = data2_10[0][1]
clips = [np.copy(d[0]) for d in data]
clips2 = [np.copy(d[0]) for d in data2]
clips_10 = [np.copy(d[0]) for d in data_10]
clips2_10 = [np.copy(d[0]) for d in data2_10]

pose_labels1 = pose_labels1[0]
pose_labels2 = pose_labels2[0]

In [18]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
pose_dim_10 = sum([feature_dims_10[feature] for feature in pose_features])

pose_dim2 = sum([feature_dims2[feature] for feature in pose_features])
pose_dim2_10 = sum([feature_dims2_10[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
target_dim = sum([feature_dims[feature] for feature in target_features])
print(phase_dim, " ", cost_dim, " ", target_dim)


8   24   48


In [21]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips])
x_tensors_10 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips_10])
y_tensors = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips])
y_tensors_10 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips_10])

x_tensors2 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips2])
x_tensors2_10 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips2_10])
y_tensors2 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips2])
y_tensors2_10 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips2_10])


pose_data1 = x_tensors[:,  :,  phase_dim:phase_dim+pose_dim]
pose_data1_10 = x_tensors_10[:,  :,  phase_dim:phase_dim+pose_dim_10]
pose_data2 = x_tensors2[:, :, phase_dim:phase_dim+pose_dim2]
pose_data2_10 = x_tensors2_10[:, :, phase_dim:phase_dim+pose_dim2_10]
pose_data = torch.cat((pose_data1, pose_data2), dim=2)

In [45]:
dataset_p = TensorDataset(pose_data, pose_data)
dataset_p1 = TensorDataset(pose_data1, pose_data1)
dataset_p2 = TensorDataset(pose_data2, pose_data2)

dataset_p1_10_C = TensorDataset(pose_data1_10, pose_data1)
dataset_p2_10_C = TensorDataset(pose_data2_10, pose_data2)
dataset_p1_10 = TensorDataset(pose_data1_10, pose_data1_10)
dataset_p2_10 = TensorDataset(pose_data2_10, pose_data2_10)

datasetR1 = TensorDataset(x_tensors, y_tensors)
datasetR1_10 = TensorDataset(x_tensors_10, y_tensors_10)
datasetR2 = TensorDataset(x_tensors2, y_tensors2)
datasetR2_10 = TensorDataset(x_tensors2_10, y_tensors2_10)

datasetR1_10_A = TensorDataset(x_tensors_10, y_tensors)
datasetR2_10_A = TensorDataset(x_tensors2_10, y_tensors2)
# datasetR3 = TensorDataset(x_tensors3, y_tensors3)
# datasetR4 = TensorDataset(x_tensors4, y_tensors4)
# datasetR1_R3 = TensorDataset(x_tensors, y_tensors3)

N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio

train_set_p, val_set_p, test_set_p = random_split(dataset_p, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p1, val_set_p1, test_set_p1 = random_split(dataset_p1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p2, val_set_p2, test_set_p2 = random_split(dataset_p2, [val_ratio*2, val_ratio, train_ratio-test_ratio], generator=torch.Generator().manual_seed(2021))

train_set_p1_10_C, val_set_p1_10_C, test_set_p1_10_C = random_split(dataset_p1_10_C, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p1_10, val_set_p1_10, test_set_p1_10 = random_split(dataset_p1_10, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p2_10_C, val_set_p2_10_C, test_set_p2_10_C = random_split(dataset_p2_10_C, [val_ratio*2, val_ratio, train_ratio-test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p2_10, val_set_p2_10, test_set_p2_10 = random_split(dataset_p2_10, [val_ratio*2, val_ratio, train_ratio-test_ratio], generator=torch.Generator().manual_seed(2021))

train_setR1, val_setR1, test_setR1 = random_split(datasetR1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2, val_setR2, test_setR2 = random_split(datasetR2, [val_ratio*2, val_ratio, train_ratio-val_ratio], generator=torch.Generator().manual_seed(2021))

train_setR1_10, val_setR1_10, test_setR1_10 = random_split(datasetR1_10, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2_10, val_setR2_10, test_setR2_10 = random_split(datasetR2_10, [val_ratio*2, val_ratio, train_ratio-val_ratio], generator=torch.Generator().manual_seed(2021))

train_setR1_10_A, val_setR1_10_A, test_setR1_10_A = random_split(datasetR1_10_A, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2_10_A, val_setR2_10_A, test_setR2_10_A = random_split(datasetR2_10_A, [val_ratio*2, val_ratio, train_ratio-val_ratio], generator=torch.Generator().manual_seed(2021))

In [24]:
def extract_targets(train_set, val_set, test_set, target_dim):
    t1, t2, t3, t33 = [], [], [], []
    for i in range(len(train_set)):
        x = train_set[i][0]
        y = train_set[i][1]
        t1.append((x[:, :-target_dim], y[:, :-target_dim]))
    for i in range(len(val_set)):
        x = val_set[i][0]
        y = val_set[i][1]
        t2.append((x[:, :-target_dim:], y[:, :-target_dim:]))
    for i in range(len(test_set)):
        x = test_set[i][0]
        y = test_set[i][1]
        t3.append((x[:, :-target_dim:], y[:, :-target_dim:]))
        t33.append((x[:, :-target_dim:], y))
    return t1, t2, t3, t33




In [46]:
t1, v1, te1, te11 = extract_targets(train_set=train_setR1, val_set=val_setR1, test_set=test_setR1, target_dim=target_dim)
t1_10, v1_10, te1_10, te11_10 = extract_targets(train_set=train_setR1_10, val_set=val_setR1_10, test_set=test_setR1_10, target_dim=target_dim)
t1_10_A, v1_10_A, te1_10_A, te11_10_A = extract_targets(train_set=train_setR1_10_A, val_set=val_setR1_10_A, test_set=test_setR1_10_A, target_dim=target_dim)
t2, v2, te2, te22 = extract_targets(train_set=train_setR2, val_set=val_setR2, test_set=test_setR2, target_dim=target_dim)
t2_10, v2_10, te2_10, te22_10 = extract_targets(train_set=train_setR2_10, val_set=val_setR2_10, test_set=test_setR2_10, target_dim=target_dim)
t2_10_A, v2_10_A, te2_10_A, te22_10_A = extract_targets(train_set=train_setR2_10_A, val_set=val_setR2_10_A, test_set=test_setR2_10_A, target_dim=target_dim)



# AE ONLY

In [67]:
ae_name1 = "AE_R1_Two_10_31J"
ae1 = MLP_v2(config=config, dimensions=[pose_dim_10],
                 pos_dim=feature_dims["pos"], rot_dim=feature_dims["rotMat2"], vel_dim=feature_dims["velocity"],
                 train_set=train_set_p1_10_C, val_set=val_set_p1_10_C, test_set=test_set_p1_10_C, name=ae_name1)

ae1.decoder = nn.Sequential(
    nn.Linear(in_features=config["k"], out_features=config["hidden_dim"]),
    nn.ELU(),nn.Dropout(p=config["keep_prob"]),
    nn.Linear(in_features=config["hidden_dim"], out_features=config["hidden_dim"]),
    nn.ELU(),
    nn.Linear(in_features=config["hidden_dim"], out_features=pose_dim),
)
# ae1 = MLP_v2(config=config, dimensions=[pose_dim_10],
#                  pos_dim=feature_dims_10["pos"], rot_dim=feature_dims_10["rotMat2"], vel_dim=feature_dims_10["velocity"],
#                  train_set=train_set_p1_10, val_set=val_set_p1_10, test_set=test_set_p1_10, name=ae_name1)

# ae1 = MLP.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.2/AE_R1_Two/0.001.256.pbz2")
# ae1.name = ae_name1

In [72]:
ae_name2 = "AE_R2_Two_10_31J"
ae2 = MLP_v2(config=config, dimensions=[pose_dim2_10],
                 pos_dim=feature_dims2["pos"], rot_dim=feature_dims2["rotMat2"], vel_dim=feature_dims2["velocity"],
                 train_set=train_set_p2_10_C, val_set=val_set_p2_10_C, test_set=test_set_p2_10_C, name=ae_name2)
#
ae2.decoder = nn.Sequential(
    nn.Linear(in_features=config["k"], out_features=config["hidden_dim"]),
    nn.ELU(),nn.Dropout(p=config["keep_prob"]),
    nn.Linear(in_features=config["hidden_dim"], out_features=config["hidden_dim"]),
    nn.ELU(),
    nn.Linear(in_features=config["hidden_dim"], out_features=pose_dim2),
)
# ae2 = MLP.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.2/AE_R5_Two/0.001.256.pbz2")
# ae2.name = ae_name2

In [41]:
def fit(model, name, version="0.1", MAX_EPOCHS=200):
    logger=TensorBoardLogger(save_dir="logs/", name=name, version=version)

    trainer = pl.Trainer(
        default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
        gpus=1, precision=16,
        min_epochs=20,
        logger=logger,
        max_epochs=MAX_EPOCHS,
        stochastic_weight_avg=True
    )
    trainer.fit(model)
    trainer.test(model)

In [59]:
fit(ae1, ae_name1)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 162 K 
1 | decoder | Sequential | 227 K 
---------------------------------------
389 K     Trainable params
0         Non-trainable params
389 K     Total params
1.559     Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7fd5d7d37850> for <torch.optim.swa_utils.SWALR object at 0x7fd5d86c69a0>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.03847884759306908,
 'ptl/test_pos_loss': 0.06162587180733681,
 'ptl/test_rot_loss': 0.029512206092476845,
 'test_loss': 0.04472292214632034}
--------------------------------------------------------------------------------


In [63]:
fit(ae2, ae_name2, MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 162 K 
1 | decoder | Sequential | 227 K 
---------------------------------------
389 K     Trainable params
0         Non-trainable params
389 K     Total params
1.559     Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7fd5d841aa60> for <torch.optim.swa_utils.SWALR object at 0x7fda6cf8e730>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.12763585150241852,
 'ptl/test_pos_loss': 0.23104818165302277,
 'ptl/test_rot_loss': 0.0874711349606514,
 'test_loss': 0.1603003591299057}
--------------------------------------------------------------------------------


In [64]:
clean_checkpoints(os.path.join(MODEL_PATH,ae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,ae_name2))

ae1.save_checkpoint(best_val_loss=0.001)
ae2.save_checkpoint(best_val_loss=0.001)


'/home/nuoc/Documents/MEX/models/version_0.2/AE_R2_Two_10_31J/0.001.256.pbz2'

In [68]:
model_name = "MLP_MoE_R1_Two_10_31"
model_name2 = "MLP_MoE_R2_Two_Reduced_10_31"

featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim_10,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2_10,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims2["posCost"],
    "rotCost":feature_dims2["rotCost"]
    }

in_slice = [phase_dim, pose_dim_10, cost_dim]
in_slice2 = [phase_dim, pose_dim2_10, cost_dim]

out_slice = [phase_dim, config["k"], cost_dim]

# temp = MLP_MIX(config=config, input_dims=[pose_dim, pose_dim_10])
# temp2 = MLP_MIX(config=config, input_dims=[pose_dim2, pose_dim2_10])

# pose_encoder = temp.active_models[1]
# pose_encoder.decoder = temp.active_models[0].decoder

pose_encoder = ae1
pose_encoder2 = ae2

# pose_encoder2 = temp2.active_models[1]
# pose_encoder2.decoder = temp2.active_models[0].decoder

# pose_encoder.encoder.load_state_dict(ae1.encoder.state_dict())
# pose_encoder.decoder.load_state_dict(ae1.decoder.state_dict())

# pose_encoder2.encoder.load_state_dict(ae2.encoder.state_dict())
# pose_encoder2.decoder.load_state_dict(ae2.decoder.state_dict())


middle_layer = torch.nn.Sequential()
model1 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1_10_A, val_set=v1_10_A, test_set=te1_10_A,
                                 name=model_name
                                   )

In [73]:
pose_encoder2 = ae2
model2 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder2, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2_10_A, val_set=v2_10_A, test_set=te2_10_A,
                                 name=model_name2
                                   )

In [70]:
fit(model1, name=model_name, version="6-31", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 389 K 
1 | middle_layer     | Sequential | 0     
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
2.9 M     Trainable params
0         Non-trainable params
2.9 M     Total params
11.497    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7fd5d7d13be0> for <torch.optim.swa_utils.SWALR object at 0x7fd5d7d26be0>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.12522226572036743,
 'ptl/test_min_pos_cost': 0.32797345519065857,
 'ptl/test_min_rot_cost': 0.1529463678598404,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.02446683496236801,
 'ptl/test_rot_loss': 0.08301495760679245,
 'ptl/test_sum_pos_cost': 3.976757526397705,
 'ptl/test_sum_rot_cost': 1.9399032592773438,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.16007418930530548}
--------------------------------------------------------------------------------


In [74]:
model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

fit(model2, name=model_name2, version="AE_ONLY", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 389 K 
1 | middle_layer     | Sequential | 0     
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
2.9 M     Trainable params
0         Non-trainable params
2.9 M     Total params
11.497    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7fd601fe0250> for <torch.optim.swa_utils.SWALR object at 0x7fdc0c4f07c0>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.13117624819278717,
 'ptl/test_min_pos_cost': -0.14672458171844482,
 'ptl/test_min_rot_cost': 0.06058218702673912,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.02856721356511116,
 'ptl/test_rot_loss': 0.07387720048427582,
 'ptl/test_sum_pos_cost': -1.5715614557266235,
 'ptl/test_sum_rot_cost': 0.7631706595420837,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.10647981613874435}
--------------------------------------------------------------------------------


In [76]:
clean_checkpoints(os.path.join(MODEL_PATH,model_name))
clean_checkpoints(os.path.join(MODEL_PATH,model_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)

In [88]:
generate_animation(model=model1, test_set=te11_10_A, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+model_name,
                   )

generate_animation(model=model2, test_set=te22_10_A, feature_dims=feature_dims2,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+model_name2,
                   )


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])


# MLP MIX - Z as input

In [29]:
mlpmix_name = "MLPMIX-Single-R1-Z-Input"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim])

mlpmix.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model

model1 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=mlpmix_name
                                   )

In [30]:
fit(model1, mlpmix_name, version="0.2",MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 454 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.281    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568cce55e0> for <torch.optim.swa_utils.SWALR object at 0x7f568c977850>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.057931140065193176,
 'ptl/test_min_pos_cost': -0.004734804853796959,
 'ptl/test_min_rot_cost': 0.21398384869098663,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.011098526418209076,
 'ptl/test_rot_loss': 0.05978814885020256,
 'ptl/test_sum_pos_cost': -0.02137945406138897,
 'ptl/test_sum_rot_cost': 2.821186065673828,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.11007595807313919}
--------------------------------------------------------------------------------


In [31]:
mlpmix_name = "MLPMIX-Single-R2-Z-Input-Reduced"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim2])

mlpmix.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=mlpmix_name
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [32]:

fit(model2, mlpmix_name, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 460 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
2.9 M     Trainable params
131 K     Non-trainable params
3.1 M     Total params
12.306    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562caec8b0> for <torch.optim.swa_utils.SWALR object at 0x7f55ee8828b0>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.0726909190416336,
 'ptl/test_min_pos_cost': -0.002268813783302903,
 'ptl/test_min_rot_cost': 0.3286762833595276,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.017275160178542137,
 'ptl/test_rot_loss': 0.0653780847787857,
 'ptl/test_sum_pos_cost': -0.007123080547899008,
 'ptl/test_sum_rot_cost': 4.181217193603516,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.09389448165893555}
--------------------------------------------------------------------------------


In [33]:
clean_checkpoints(os.path.join(MODEL_PATH,"MLPMIX-Single-R1-Z-Input"))
clean_checkpoints(os.path.join(MODEL_PATH,mlpmix_name))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [34]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/MLPMIX-Single-R1-Z-Input")
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/MLPMIX-Single-R2-Z-Input-Reduced")

torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# MLP MIX - Z Concat

In [35]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
mlpmix_name = "MLPMIX-Single-R1-Z-Concat"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim])

mlpmix.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model

model1 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=mlpmix_name
                                   )

In [36]:
fit(model1, mlpmix_name, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 454 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 3.0 M 
------------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.378    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562ca5aa90> for <torch.optim.swa_utils.SWALR object at 0x7f562c9dd400>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.11958379298448563,
 'ptl/test_min_pos_cost': -0.005060994066298008,
 'ptl/test_min_rot_cost': 0.20295865833759308,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.01100150402635336,
 'ptl/test_rot_loss': 0.058779697865247726,
 'ptl/test_sum_pos_cost': -0.0016244666185230017,
 'ptl/test_sum_rot_cost': 2.7480456829071045,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1672605574131012}
--------------------------------------------------------------------------------


In [37]:
mlpmix_name = "MLPMIX-Single-R2-Z-Concat-Reduced"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim2])

mlpmix.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=mlpmix_name
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [38]:

fit(model2, mlpmix_name, version="0.2", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 460 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 3.0 M 
------------------------------------------------
3.5 M     Trainable params
131 K     Non-trainable params
3.6 M     Total params
14.403    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f55f0e0e3d0> for <torch.optim.swa_utils.SWALR object at 0x7f562c79c880>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.1670786738395691,
 'ptl/test_min_pos_cost': -0.0012020673602819443,
 'ptl/test_min_rot_cost': 0.31904318928718567,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.020937127992510796,
 'ptl/test_rot_loss': 0.07685936987400055,
 'ptl/test_sum_pos_cost': 0.008795861154794693,
 'ptl/test_sum_rot_cost': 4.102316856384277,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.2700470983982086}
--------------------------------------------------------------------------------


In [75]:
clean_checkpoints(os.path.join(MODEL_PATH,"MLPMIX-Single-R1-Z-Concat"))
clean_checkpoints(os.path.join(MODEL_PATH,mlpmix_name))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


NameError: name 'mlpmix_name' is not defined

In [40]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/MLPMIX-Single-R1-Z-Concat")
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+mlpmix_name)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# RBF - Z as input

In [41]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
rbf_name1 = "RBF-Single-R1-Z-Input"
rbf = RBF(config=config, input_dims=[pose_dim])

rbf.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model

model1 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=rbf_name1
                                   )

In [42]:
fit(model1, rbf_name1, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
12.018    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568cdd7a30> for <torch.optim.swa_utils.SWALR object at 0x7f568ce70550>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.09364240616559982,
 'ptl/test_min_pos_cost': -0.0053687128238379955,
 'ptl/test_min_rot_cost': 0.1818535178899765,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.020856985822319984,
 'ptl/test_rot_loss': 0.07930824160575867,
 'ptl/test_sum_pos_cost': 0.008359869010746479,
 'ptl/test_sum_rot_cost': 2.6449549198150635,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1203242689371109}
--------------------------------------------------------------------------------


In [43]:
rbf_name2 = "RBF-Single-R2-Z-Input-Reduced"
rbf = RBF(config=config, input_dims=[pose_dim2])

rbf.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=rbf_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [44]:
fit(model2, rbf_name2, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
2.9 M     Trainable params
65.8 K    Non-trainable params
3.0 M     Total params
12.043    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562c9ab670> for <torch.optim.swa_utils.SWALR object at 0x7f562c9a5f10>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.10660167038440704,
 'ptl/test_min_pos_cost': 0.0022916956804692745,
 'ptl/test_min_rot_cost': 0.3098534047603607,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.024783343076705933,
 'ptl/test_rot_loss': 0.09240834414958954,
 'ptl/test_sum_pos_cost': 0.077973872423172,
 'ptl/test_sum_rot_cost': 4.071671962738037,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.16455666720867157}
--------------------------------------------------------------------------------


In [45]:
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name1))
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [46]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name2)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# RBF - Z Concat

In [47]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
rbf_name1 = "RBF-Single-R1-Z-Concat"
rbf = RBF(config=config, input_dims=[pose_dim])

rbf.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model

model1 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=rbf_name1
                                   )

In [49]:
fit(model1, rbf_name1, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
14.115    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568cddbd60> for <torch.optim.swa_utils.SWALR object at 0x7f568d11b550>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.10979676246643066,
 'ptl/test_min_pos_cost': -0.0033021767158061266,
 'ptl/test_min_rot_cost': 0.207255020737648,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.009942011907696724,
 'ptl/test_rot_loss': 0.05450308322906494,
 'ptl/test_sum_pos_cost': 0.009184036403894424,
 'ptl/test_sum_rot_cost': 2.807267904281616,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.17189908027648926}
--------------------------------------------------------------------------------


In [50]:
rbf_name2 = "RBF-Single-R2-Z-Concat-Reduced"
rbf = RBF(config=config, input_dims=[pose_dim2])

rbf.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=rbf_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [51]:

fit(model2, rbf_name2, version="0.2", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.5 M     Trainable params
65.8 K    Non-trainable params
3.5 M     Total params
14.140    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562c76aca0> for <torch.optim.swa_utils.SWALR object at 0x7f55ec536610>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.15703630447387695,
 'ptl/test_min_pos_cost': -0.0006652036099694669,
 'ptl/test_min_rot_cost': 0.3239959478378296,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.020156705752015114,
 'ptl/test_rot_loss': 0.07347622513771057,
 'ptl/test_sum_pos_cost': 0.030438141897320747,
 'ptl/test_sum_rot_cost': 4.198546409606934,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.175152987241745}
--------------------------------------------------------------------------------


In [52]:
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name1))
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [53]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name2)



torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# VAE - Z as input

In [54]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
vae_name1 = "VAE-Single-R1-Z-Input"
vae = VAE(config=config, input_dims=[pose_dim])

vae.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model

model1 = MotionGenerationModel_VAE(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=vae_name1
                                   )

In [55]:
fit(model1, vae_name1, MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.281    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f55e9cdc760> for <torch.optim.swa_utils.SWALR object at 0x7f569788db50>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 0.004071646369993687,
 'ptl/test_loss': 0.08888862282037735,
 'ptl/test_min_pos_cost': -0.004451391752809286,
 'ptl/test_min_rot_cost': 0.2351878434419632,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.019994894042611122,
 'ptl/test_recon_loss': 0.08481697738170624,
 'ptl/test_rot_loss': 0.0779380202293396,
 'ptl/test_sum_pos_cost': -0.024823816493153572,
 'ptl/test_sum_rot_cost': 2.9984066486358643,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.12005238980054855}
--------------------------------------------------------------------------------


In [56]:
vae_name2 = "VAE-Single-R2-Z-Input-Reduced"
vae = VAE(config=config, input_dims=[pose_dim2])

vae.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_VAE(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=vae_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [57]:
fit(model2, vae_name2, MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
2.9 M     Trainable params
131 K     Non-trainable params
3.1 M     Total params
12.306    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f5624ad97f0> for <torch.optim.swa_utils.SWALR object at 0x7f568c7ce190>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 0.002039763145148754,
 'ptl/test_loss': 0.11052046716213226,
 'ptl/test_min_pos_cost': 0.001397526590153575,
 'ptl/test_min_rot_cost': 0.32723695039749146,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.02676679566502571,
 'ptl/test_recon_loss': 0.10848070681095123,
 'ptl/test_rot_loss': 0.09787915647029877,
 'ptl/test_sum_pos_cost': 0.05337149649858475,
 'ptl/test_sum_rot_cost': 4.250061988830566,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1679312139749527}
--------------------------------------------------------------------------------


In [58]:
clean_checkpoints(os.path.join(MODEL_PATH,vae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,vae_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [59]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,use_vae=True,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,use_vae=True,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name2)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# VAE - Z Concat

In [62]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
vae_name1 = "VAE-Single-R1-Z-Concat"
vae = VAE(config=config, input_dims=[pose_dim])

vae.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model

model1 = MotionGenerationModel_VAE_ZZ(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=vae_name1
                                   )

In [63]:
fit(model1, vae_name1, MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.378    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568c977850> for <torch.optim.swa_utils.SWALR object at 0x7f562c84ec10>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 2.144406062143389e-05,
 'ptl/test_loss': 0.04409467428922653,
 'ptl/test_min_pos_cost': -0.004690519068390131,
 'ptl/test_min_rot_cost': 0.22766542434692383,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.009413277730345726,
 'ptl/test_recon_loss': 0.044073231518268585,
 'ptl/test_rot_loss': 0.05002091825008392,
 'ptl/test_sum_pos_cost': -0.02753099799156189,
 'ptl/test_sum_rot_cost': 2.9506356716156006,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.07824178040027618}
--------------------------------------------------------------------------------


In [64]:
vae_name2 = "VAE-Single-R2-Z-Concat-Reduced"
vae = VAE(config=config, input_dims=[pose_dim2])

vae.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_VAE_ZZ(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=vae_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [65]:

fit(model2, vae_name2, MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.5 M     Trainable params
131 K     Non-trainable params
3.6 M     Total params
14.403    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568c986580> for <torch.optim.swa_utils.SWALR object at 0x7f55ee969310>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 0.00015117238217499107,
 'ptl/test_loss': 0.0905328169465065,
 'ptl/test_min_pos_cost': -0.0015516206622123718,
 'ptl/test_min_rot_cost': 0.34485024213790894,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.022057222202420235,
 'ptl/test_recon_loss': 0.09038165211677551,
 'ptl/test_rot_loss': 0.07953483611345291,
 'ptl/test_sum_pos_cost': -0.003954050596803427,
 'ptl/test_sum_rot_cost': 4.304420471191406,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1030849739909172}
--------------------------------------------------------------------------------


In [66]:
clean_checkpoints(os.path.join(MODEL_PATH,vae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,vae_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [67]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,use_vae=True,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,use_vae=True,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name2)





torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


In [86]:
def setVec3(struct, vec):
    struct["x"] = vec[0].item()
    struct["y"] = vec[1].item()
    struct["z"] = vec[2].item()

def setVec6(struct, vec):
    for r, cell in enumerate(["x", "y", "z"]):
        for col, column in enumerate(["c0", "c1"]):
            struct[column][cell] = vec[r, col].item()

def insert_pos(template,
               positions=None, rotations=None, velocity=None,
               tPos=None, tRot=None, name="Replay", level=0):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            t = 0
            for j in range(shape[2]):
                jo = template["frames"][f]["joints"][j]
                if level >= jo["level"]:
                    if positions is not None:
                        setVec3(jo["position"], positions[c,f,j])
                    if rotations is not None:
                        setVec3(jo["velocity"], velocity[c,f,j])
                    if velocity is not None:
                        setVec6(jo["rotMat"], rotations[c,f,j])


                    if jo["key"]:
                        if tPos is not None:
                            setVec3(jo["cost"]["TargetPosition"], tPos[c,f,t])
                        if tRot is not None:
                            setVec6(jo["cost"]["TargetRotation"], tRot[c,f,t])
                        t+=1
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

def generate_animation(model, test_set, feature_dims, template_path, output_path, use_vae=False, n=5, level=0):
    idx = np.arange(n)
    with torch.no_grad():
        model.eval()
        model.cpu()

        x = torch.stack([test_set[i][0] for i in idx])
        y = torch.stack([test_set[i][1] for i in idx])
        shape =x.shape
        x = x.view(-1, shape[-1])

        x.cpu()
        if use_vae:
            out, z, mu, logvar = model(x)
        else:
            out = model(x)
        x_c = torch.cat(out,dim=1).detach()
        generated = x_c
        generated = generated.view(n, shape[1], -1)

    phase= feature_dims["phase_vec_l2"]
    toPosDim = phase+feature_dims["pos"]
    toRotDim = toPosDim + feature_dims["rotMat2"]
    toVelDim = toRotDim + feature_dims["velocity"]

    gPos = generated[:, :, phase:toPosDim]
    gRot = generated[:, :, toPosDim:toRotDim]
    gVel = generated[:, :, toRotDim:toVelDim]

    oPos = y[:, :, phase:toPosDim]
    oRot = y[:, :, toPosDim:toRotDim]
    oVel = y[:, :, toRotDim:toVelDim]

    tPos = y[:, :, -target_dim:-target_dim+3*4]
    tRot = y[:, :, -target_dim+3*4:]

    print(gPos.shape, gRot.shape, gVel.shape)
    print(oPos.shape, oRot.shape, oVel.shape)

    clip_length = gPos.shape[1]
    gPos_r = gPos.reshape((n, clip_length, -1, 3))
    gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
    gVel_r = gVel.reshape((n, clip_length, -1, 3))

    oPos_r = oPos.reshape((n, clip_length, -1, 3))
    oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))
    oVel_r = oVel.reshape((n, clip_length, -1, 3))

    tPos_r = tPos.reshape((n, clip_length, -1, 3))
    tRot_r = tRot.reshape((n, clip_length, -1, 3, 3))

    template = js.load(open(template_path))

    insert_pos(template, oPos_r, oRot_r, oVel_r, tPos_r, tRot_r,output_path+"_O", level=level)
    insert_pos(template, gPos_r, gRot_r, gVel_r, tPos_r, tRot_r,output_path+"_G", level=level)
